In [120]:
import pandas as pd
import numpy as np
from random import randint

## acquire data
train_df = pd.read_csv('train.csv')
gender_df = pd.read_csv('gender_submission.csv')
test_df = pd.read_csv('test.csv')

print(train_df.head())
#print(test_df.info())
#print(gender_df.info())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [121]:
## check what data is impactful

## try Embarked
## there are some Embarked values missing, add with the most frequent
embarked_fill = train_df['Embarked'].mode()
#print(embarked_fill)

embarked_null = train_df['Embarked'].isnull().sum()

for b in range (0,embarked_null):
    train_df['Embarked'].fillna(embarked_null, inplace = True)

train = (train_df[['Embarked','Survived']].groupby(["Embarked"]).mean())

#print(train_df.info())
#print(train)

In [122]:
## try Pclass
train = (train_df[['Pclass','Survived']].groupby(["Pclass"]).mean())
## findings to be impactful: Pclass

In [123]:
## try Gender
train = (train_df[['Sex','Survived']].groupby(["Sex"]).mean())
## findings to be impactful: Pclass, Sex, Embarked

In [124]:
## try Age

## age has several data missing, need to fill
## null age + mean age + std deviation age // or use interpolate()
not_null_age = train_df['Age'].count()
null_age = train_df['Age'].isnull().sum()

std_age = train_df['Age'].std()
avg_age = train_df['Age'].mean()
summation = int(avg_age+std_age)
diff = int(avg_age-std_age)

for a in range(0,null_age):
    rand_age = randint(diff,summation)
    #rand_age = train_df['Age'].interpolate()
    train_df['Age'].fillna(rand_age,inplace = True)
    test_df['Age'].fillna(rand_age,inplace=True)

   
train_df['age_map'] = pd.cut(train_df['Age'],4,labels= [3,2,1,0])
cat_age = train_df['age_map']
test_df['age_map'] = pd.cut(test_df['Age'],4,labels=[3,2,1,0])
train = (train_df[['Age','Survived']].groupby([cat_age]).mean()) 
#print(train_df.info())
#print(test_df.info())
print(train_df.head(5))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked age_map  
0      0         A/5 21171   7.2500   NaN        S       2  
1      0          PC 17599  71.2833   C85        C       2  
2      0  STON/O2. 3101282   7.9250   NaN        S       2  
3      0            113803  53.1000  C123        S       2  
4      0       

In [125]:
#FARE
# FARE missing 1 data in test_df, so add one with the most frequent value

fare_frequent = test_df['Fare'].mean(skipna = True)

test_df['Fare'].fillna(0,inplace = True) # Accuracy:  82.38
#test_df['Fare'].fillna(fare_frequent,inplace = True) # Accuracy:  82.04

test_df['Fare'] = pd.cut(test_df['Fare'],4,  labels=[0,1,2,3])
train_df['Fare'] = pd.cut(train_df['Fare'],4, labels=[0,1,2,3])

#print(test_df.info())

In [126]:
# IF THE PERSON IS ALONE (without SibSp and Parch)
train_df['number_of_family'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['number_of_family'] = test_df['SibSp'] + train_df['Parch'] + 1

train_df['number_of_family'].loc[train_df['number_of_family']==1] = 1
train_df['number_of_family'].loc[train_df['number_of_family']>1]  = 0
test_df['number_of_family'].loc[test_df['number_of_family']==1] = 1
test_df['number_of_family'].loc[test_df['number_of_family']>1]  = 0

# train = (train_df[['number_of_family','Survived']].groupby('number_of_family', as_index=False).mean())
print(train_df.head(5))


C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket Fare Cabin Embarked age_map  number_of_family  
0      0         A/5 21171    0   NaN        S       2                 0  
1      0          PC 17599    0   C85        C       2                 0  
2      0  STON/O2. 3101282    0   NaN        S       2                 1  
3      0            

In [127]:
## findings to be impactful: Pclass, Gender, Embarked, Age, Fare
# now mapping the data. turn everything to numbers
train_df['Sex'] = train_df['Sex'].replace({'female': 1, 'male': 0})
test_df['Sex'] = test_df['Sex'].replace({'female':1,'male':0})
train_df['Embarked'] = train_df['Embarked'].replace({'S':0,'Q':1,'C':2})
test_df['Embarked'] = test_df['Embarked'].replace({'S':0,'Q':1,'C':2})
#cat_age = age map
train_df['Age']=train_df['age_map']
test_df['Age'] = test_df['age_map']

In [128]:
#list of columns to drop; 
drop_columns = ['Name','SibSp', 'Parch', 'Cabin', 'Ticket','Fare']

train_df = train_df.drop(drop_columns,axis=1)
train_df = train_df.drop('PassengerId',axis=1)
train_df = train_df.drop('age_map',axis=1)

test_df = test_df.drop(drop_columns,axis=1)
test_df = test_df.drop('age_map',axis=1)
print(test_df.head(5))


   PassengerId  Pclass  Sex Age  Embarked  number_of_family
0          892       3    0   2         1               1.0
1          893       3    1   1         0               0.0
2          894       2    0   0         1               1.0
3          895       3    0   2         0               1.0
4          896       3    1   2         0               0.0


In [129]:
train_X = train_df.drop('Survived',axis=1)
train_Y = train_df['Survived']
test_X = test_df.drop("PassengerId", axis=1).copy()
print(test_X.head(5))

   Pclass  Sex Age  Embarked  number_of_family
0       3    0   2         1               1.0
1       3    1   1         0               0.0
2       2    0   0         1               1.0
3       3    0   2         0               1.0
4       3    1   2         0               0.0


In [130]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_X,train_Y)
pred_Y = decision_tree.predict(test_X)
accuracy = round(decision_tree.score(train_X,train_Y)*100,2)
print("Train Accuracy: ",accuracy)

Train Accuracy:  83.16


In [ ]:
# CREATE CSV with results
submission = pd.DataFrame({"PID": test_df["PassengerId"],"Survived": pred_Y})
submission.to_csv('submission.csv')